In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('raw/daily.csv')
df = df[['date','state','positive','negative']]

In [3]:
states = []
for state in df['state']:
    states.append(state)
states = list(set(states))
cleanedDf = pd.DataFrame(dtype=str)

In [4]:
for state in states:
    stateDf = df[df['state'] == state]
    
    pop = pd.read_csv('../countydemographics/county_demographics.csv')
    pop = pop[pop['state_code'] == state]
    pop = pop['population'].sum()
    
    if pop != 0:
        dates, pos, neg, rates = [], [], [], []
        for row in stateDf.iterrows():
            row = row[1]
            date = row['date']
            date = list(str(date))
            date.insert(4, '-')
            date.insert(7, '-')
            date = ''.join(date)
        
            state = row['state']
            positive = row['positive']
            negative = row['negative']
        
            if (positive+negative) != 0:
                positiveTr = positive / (positive + negative)
            else:
                positiveTr = 0.

            positiveTrCapita = positiveTr / pop

            dates.append(date)
            pos.append(positive)
            neg.append(negative)
            rates.append(positiveTrCapita)
        
        temp = pd.DataFrame(columns=['state_code', *dates])
        temp = temp.append(pd.Series(rates, index=dates), ignore_index=True)
        temp['state_code'] = state
    
        cleanedDf = cleanedDf.append(temp)

In [5]:
cleanedDf = cleanedDf.reindex(sorted(cleanedDf.columns), axis=1)
firstCol = cleanedDf.pop('state_code')
cleanedDf.insert(0, 'state_code', firstCol)
cleanedDf = cleanedDf.fillna(0)

In [6]:
cleanedDf = cleanedDf.reset_index()
cleanedDf = cleanedDf.drop(['index'], axis=1)

In [7]:
cleanedDf.to_csv('testing_rates_state.csv')

In [8]:
# County

In [9]:
df = pd.read_csv('../countydemographics/county_demographics.csv')
cleanedDf = pd.DataFrame(dtype=str)

In [10]:
for row in df.iterrows():
    row = row[1]
    fips = row['fips']
    state_code = row['state_code']
    pop = row['population']
    
    stateTr = pd.read_csv('testing_rates_state.csv')
    stateTr = stateTr[stateTr['state_code'] == state_code]
    stateTr = stateTr.rename({'state_code':'fips'}, axis=1)
    stateTr = stateTr * pop
    stateTr['fips'] = fips
    
    cleanedDf = cleanedDf.append(stateTr)

In [11]:
cleanedDf = cleanedDf.fillna(0)
cleanedDf = cleanedDf.reset_index()
cleanedDf = cleanedDf.drop(['index'], axis=1)
cleanedDf = cleanedDf.drop(cleanedDf.columns[0],axis=1)

In [12]:
cleanedDf.to_csv('testing_rates_county.csv')